Student's Name and Email Address

Boise State University, Department of Chemistry and Biochemistry

## CHEM 324: PChem Lab {-}
# Worksheet 3: Computational Thermochemistry of Alkenes {-}

In [ ]:
# @title PySCF Setup { display-mode: "form" }
# Import the main components of PySCF used in this worksheet
!pip install pyscf
!pip install pyberny
!pip install pyscf\[geomopt\]
from pyscf import gto, scf, dft, mp, cc
from pyscf.geomopt.berny_solver import optimize
#
from scipy.constants import physical_constants # we will need these for units conversion

In [ ]:
# @title Notebook Setup { display-mode: "form" }
# Import the main modules used in this worksheet
import numpy as np
import matplotlib.pyplot as plt
# Load the google drive with your files
from google.colab import drive
drive.mount('/content/drive')
# The following needs to be the path of the folder with all your datafile in .csv format
base_path = '/content/drive/MyDrive/'

In [ ]:
# @title Set Local Path { display-mode: "form" }
# The following needs to be the path of the folder with all your collected data in .csv format
local_path="Colab Notebooks/CompThermo_Data/" # @param {type:"string"}
path = base_path+local_path

In [ ]:
# @title Install and load RDKit, CirPy, and Py3DMol { display-mode: "form" }
!pip install rdkit
from rdkit import Chem
from rdkit.Chem import Draw
!pip install cirpy
import cirpy
! pip install py3Dmol
import py3Dmol

In [ ]:
# @title Utilities { display-mode: "form" }
def mol_to_xyz(mol, comment=""):
    bohr_to_ang = 0.529177210903
    coords = mol.atom_coords() * bohr_to_ang
    symbols = [mol.atom_symbol(i) for i in range(mol.natm)]

    lines = []
    lines.append(str(mol.natm))
    lines.append(comment)
    for sym, (x, y, z) in zip(symbols, coords):
        lines.append(f"{sym:2s} {x: .8f} {y: .8f} {z: .8f}")

    return "\n".join(lines)

## Task 1: Select a Model Chemistry {-}

**Introduction** 

When starting a computational experiment one needs to perform a preliminary assessment of the accuracy and costs of different levels of theory for the specific tasks at hand. For this preliminary calibration experiment, the goal is to figure out a model chemistry (i.e. a combination of method and basis set) that gives reasonable stability trends for alkenes and that requires a reasonable computational time. This task is vague on purpose, as different researchers with different goals and resources will have different definitions of what is reasonable. It is important in your selection process that you provide some justification on why something is (or is not) reasonable to you. 

Usually the model testing and model selection parts of the experiment are performed on simpler systems, as for these it is either easier to find experimental literature data (i.e. we can use experimental benchmarks) or it is feasible to perform very accurate simulations (i.e. we can compare simpler model with theoretical benchmarks). For this lab, we will perform the first three sets of experiments on smaller alkenes with formula C4H8. 

**Experiments** 

For each isomer of your system, generate the coordinates using CirPy and perform `single-point energy` calculations using different model chemistries. 
In particular you should consider different combinations of the following methods:

* Hartree-Fock (HF)
* Density Functional Theory (DFT) with the PBE functional
* Density Functional Theory (DFT) with the hybrid B3LYP functional
* Moeller-Plesset Second Order Perturbation Theory (MP2)
* Couple-Cluster Singles and Doubles (CCSD)
* Couple-Cluster Singles, Doubles, and perturbative Triples (CCSD(T))

with the following basis sets:

* Pople double-zeta 6-31G
* Pople double-zeta with polarization and diffuse 6-31+G*
* Pople triple-zeta 6-311G
* Pople triple-zeta with polarization and diffuse 6-311+G*
* Weigend–Ahlrichs double-zeta minimal def2-SV(P) 
* Weigend–Ahlrichs double-zeta with polarization def2-SVP 
* Weigend–Ahlrichs double-zeta with polarization and diffuse def2-SVPD 
* Weigend–Ahlrichs triple-zeta def2-TZVP
* Weigend–Ahlrichs triple-zeta with diffuse def2-TZVPD
* Dunning correlation-consistent double-zeta cc-PVDZ
* Dunning correlation-consistent augmented double-zeta aug-cc-PVDZ
* Dunning correlation-consistent triple-zeta cc-PVTZ
* Dunning correlation-consistent augmented triple-zeta aug-cc-PVTZ


While the theory behind methods and basis sets is beyond the goals of this lab, here are some general principles: as you climb up the ladder of levels of theory, your results will account for more electron correlation effects. As the basis set becomes larger, including more functions, the results will become more accurate. 

There are different strategies to test and select a model chemistry, for this task you can use the following approach: 

1. Consider one level of theory (e.g. HF) and check the effects of the basis set. 
2. Consider one basis set (e.g. 6-31G) and check the effects of the level of theory. 

For both screenings, compare the relative formation energy of different isomers with the results obtained with the most accurate approaches (highest level of theory and/or largest basis set). Compare the timings of the different calculations. Add more screening calculations or try more model chemistry as you consider appropriate. If time allows it, try to perform the most accurate calculations that you can, to use as absolute benchmarks, but be aware that some of the calcualtions may take hours to finish or may crash your Collab session. 

By the end of these screenings you should pick at least two model chemistries: one that is very accurate and slightly more expensive than ideal and one that is cheap but not too bad in terms of accuracy. In making your selection try to consider these points:
* Is the qualitative order of total energies of the different isomers consistent with the most accurate results?
* Compare with the most accurate results, is the error in the total energies comparable or larger than thermal energy? is the error comparable or larger than 1 kcal/mol (often referred to as chemical accuracy)?
* Keep in mind that relative energies (i.e. the difference in energy between two isomers) are significantly more important to get right than absolute energies
* In order to understand timings, it is useful to know the size of the basis set and the scaling of the method (HF/DFT are cubic, MP2 is order 5, CC is order 6).

## Task 2: Formation Energies {-}

**Target**

For the first experiments we will try to predict the relative stability of the different isomers of butene by looking at their formation energies. Formation energies can be defined as the total energy change associated with taking the neutral isolated atoms infinitely apart and bringing them together to form the given molecule. Alternative more experimentally acccessible definitions of formation energies involve the atoms in their stable elemental forms as reference precursors. However, for the problem at hand either the isolated atoms or the elemental precursors will have the same identical energies for all isomers. If we neglect zero-point motion contributions (vibrations) and entropic effects, comparing formation energies is equivalent to comparing the total electronic energies of the different isomers in their equilibrium geometries.

**Experiments** 

For each alkene with formula C4H8, generate the coordinates using CirPy and perform `geometry optimization` calculations using a couple of different model chemistries, as determined in the previous task. Compare the final geometries and final electronic energies and determine if the order of stability is the one expected from your organic chemistry intuition. Comment on the results and, if available, compare your relative stabilities with reference experimental values.

## Task 3: Reactivity Towards Halogenation {-}

**Target**

For the second experiments we will try to predict the stability of the different isomers of butene by comparing their reactivity with hydrohalic acids (HCl, HBr, HI). According to the Hammond postulate, alkene protonation is an endothermic step whose transition state resembles the carbocation, so alkenes that form lower-energy carbocations (relative to the alkene) react faster with hydrohalic acids. 

**Experiments** 

For each isomer of your system, generate the coordinates using CirPy and perform `geometry optimization` calculations on carbocatinos using the same model chemistries used in the previous task. Compare your results with your organic chemistry intuition and comment on the results. 

## Task 4: Predictions on More Realistic Systems {-}

**Target**

In the last experiment we will use the approaches of the previous tasks and apply them to a more complex target, namely the isomers of hexene with formula C6H12. Each of you will be randomly assigned one isomer. 

**Experiments** 

1. Before running any calculation, make a prediction based on your organic chemistry intuition. How stable is your isomer compare to all of the others? How reactive is it with hydrohalic acids? Assign your isomer a ranking in quartiles or, if you are very confident, its exact order.
2. Using B3LYP/def2-TZVP perform the geometry optimization calculations of the alkene and of the corresponding carbocation(s) (NOTE: there could be more than one, depending on your structure). 
3. Starting from the optimized geometry perform single-point energy calculations using CCSD/augccPVTZ. 
4. Compare your results with the ones from the other students and comment on the rankings and the quality of your predictions.